In [ ]:
# ==========================================
# FEATURE ENGINEERING OPTIMIZATION GUIDE
# Tambahkan code ini SETELAH merge selesai
# ==========================================

import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("🔧 Starting Feature Engineering Optimization...")

# Load dataset hasil merge Anda
app_train_fe = pd.read_csv("app_train_with_features_cleaned1.csv")
print(f"Original dataset shape: {app_train_fe.shape}")

# ==========================================
# STEP 1: ADVANCED FEATURE ENGINEERING
# ==========================================
print("\n📊 STEP 1: Creating Advanced Features...")

# 1.1 External Source Combinations (PRIORITAS TINGGI)
print("Creating External Source features...")
ext_source_cols = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']

# Pastikan kolom ada
for col in ext_source_cols:
    if col not in app_train_fe.columns:
        print(f"Warning: {col} not found, creating dummy column")
        app_train_fe[col] = np.nan

# Kombinasi External Sources
app_train_fe['EXT_SOURCE_MEAN'] = app_train_fe[ext_source_cols].mean(axis=1)
app_train_fe['EXT_SOURCE_STD'] = app_train_fe[ext_source_cols].std(axis=1)
app_train_fe['EXT_SOURCE_MAX'] = app_train_fe[ext_source_cols].max(axis=1)
app_train_fe['EXT_SOURCE_MIN'] = app_train_fe[ext_source_cols].min(axis=1)

# Weighted combination (berdasarkan korelasi dengan target dari EDA Anda)
app_train_fe['EXT_SOURCE_WEIGHTED'] = (app_train_fe['EXT_SOURCE_1'] * 0.4 +
                                      app_train_fe['EXT_SOURCE_2'] * 0.35 +
                                      app_train_fe['EXT_SOURCE_3'] * 0.25)

# Missing indicators untuk External Sources
for i in [1, 2, 3]:
    app_train_fe[f'EXT_SOURCE_{i}_MISSING'] = app_train_fe[f'EXT_SOURCE_{i}'].isnull().astype(int)

print("✅ External Source features created")

# 1.2 Age-based Features
print("Creating age-based features...")
if 'DAYS_BIRTH' in app_train_fe.columns:
    app_train_fe['YEARS_BIRTH'] = app_train_fe['DAYS_BIRTH'] / -365.25

    # Age groups berdasarkan insight bisnis Indonesia
    app_train_fe['AGE_GROUP'] = pd.cut(app_train_fe['YEARS_BIRTH'],
                                      bins=[0, 25, 35, 45, 55, 100],
                                      labels=['Young', 'Adult', 'Middle', 'Senior', 'Elder'])

    # Employment stability
    if 'DAYS_EMPLOYED' in app_train_fe.columns:
        app_train_fe['YEARS_EMPLOYED'] = app_train_fe['DAYS_EMPLOYED'] / -365.25
        app_train_fe['EMPLOYMENT_STABILITY'] = app_train_fe['YEARS_EMPLOYED'] / app_train_fe['YEARS_BIRTH']

        # Handle infinite values
        app_train_fe['EMPLOYMENT_STABILITY'] = app_train_fe['EMPLOYMENT_STABILITY'].replace([np.inf, -np.inf], np.nan)
        app_train_fe['EMPLOYMENT_STABILITY'] = app_train_fe['EMPLOYMENT_STABILITY'].fillna(0)

print("✅ Age-based features created")

# 1.3 Income-based Features
print("Creating income-based features...")
if 'AMT_INCOME_TOTAL' in app_train_fe.columns and 'CNT_FAM_MEMBERS' in app_train_fe.columns:
    app_train_fe['INCOME_PER_FAMILY_MEMBER'] = app_train_fe['AMT_INCOME_TOTAL'] / (app_train_fe['CNT_FAM_MEMBERS'] + 1)

# Income categories berdasarkan distribusi dari EDA Anda
if 'AMT_INCOME_TOTAL' in app_train_fe.columns:
    app_train_fe['INCOME_CATEGORY'] = pd.cut(app_train_fe['AMT_INCOME_TOTAL'],
                                            bins=[0, 100000, 200000, 300000, np.inf],
                                            labels=['Low', 'Medium', 'High', 'Very_High'])

print("✅ Income-based features created")

# 1.4 Interaction Features (High Impact)
print("Creating interaction features...")

# Gender-Car interaction (berdasarkan EDA Anda)
if 'CODE_GENDER' in app_train_fe.columns and 'FLAG_OWN_CAR' in app_train_fe.columns:
    app_train_fe['GENDER_CAR_INTERACTION'] = app_train_fe['CODE_GENDER'].astype(str) + '_' + app_train_fe['FLAG_OWN_CAR'].astype(str)

# Income-Credit interaction
if 'AMT_INCOME_TOTAL' in app_train_fe.columns and 'AMT_CREDIT' in app_train_fe.columns:
    app_train_fe['INCOME_CREDIT_INTERACTION'] = app_train_fe['AMT_INCOME_TOTAL'] * app_train_fe['AMT_CREDIT'] / 1e12  # Scale down

# Age-Income interaction
if 'YEARS_BIRTH' in app_train_fe.columns and 'AMT_INCOME_TOTAL' in app_train_fe.columns:
    app_train_fe['AGE_INCOME_INTERACTION'] = app_train_fe['YEARS_BIRTH'] * app_train_fe['AMT_INCOME_TOTAL'] / 1e6  # Scale down

print("✅ Interaction features created")

# 1.5 Risk Score Engineering
print("Creating composite risk scores...")

def create_risk_score(row):
    """Composite risk score berdasarkan findings EDA Anda"""
    score = 0

    # Gender risk (Male lebih berisiko dari EDA Anda)
    if row.get('CODE_GENDER') == 'M':
        score += 1

    # Contract type risk (Cash loans lebih berisiko)
    if row.get('NAME_CONTRACT_TYPE') == 'Cash loans':
        score += 1

    # Payment burden risk
    if pd.notna(row.get('PAYMENT_TO_INCOME_RATIO')) and row.get('PAYMENT_TO_INCOME_RATIO', 0) > 0.4:
        score += 2

    # Age risk (older customers lebih berisiko dari EDA)
    if pd.notna(row.get('YEARS_BIRTH')) and row.get('YEARS_BIRTH', 0) > 50:
        score += 1

    # External source missing risk
    if pd.isna(row.get('EXT_SOURCE_1')) and pd.isna(row.get('EXT_SOURCE_2')):
        score += 2

    # Car ownership (slight risk difference dari EDA)
    if row.get('FLAG_OWN_CAR') == 'N':
        score += 0.5

    return score

app_train_fe['COMPOSITE_RISK_SCORE'] = app_train_fe.apply(create_risk_score, axis=1)

# Regional risk score
if 'REGION_RATING_CLIENT' in app_train_fe.columns and 'REGION_RATING_CLIENT_W_CITY' in app_train_fe.columns:
    app_train_fe['REGION_RISK_SCORE'] = (app_train_fe['REGION_RATING_CLIENT'] +
                                        app_train_fe['REGION_RATING_CLIENT_W_CITY']) / 2

print("✅ Risk scores created")

print(f"After advanced feature engineering: {app_train_fe.shape}")

# ==========================================
# STEP 2: MULTICOLLINEARITY ANALYSIS & FEATURE SELECTION
# ==========================================
print("\n🔍 STEP 2: Multicollinearity Analysis...")

# Identify numerical columns (exclude IDs and target)
numerical_cols = app_train_fe.select_dtypes(include=[np.number]).columns.tolist()
exclude_cols = ['SK_ID_CURR', 'TARGET'] + [col for col in numerical_cols if 'ID' in col.upper()]
numerical_cols = [col for col in numerical_cols if col not in exclude_cols]

print(f"Analyzing {len(numerical_cols)} numerical features for multicollinearity...")

# Calculate VIF untuk subset features (VIF computation expensive untuk 200+ features)
def calculate_vif_batch(df, features, batch_size=50):
    """Calculate VIF in batches to handle large feature sets"""
    high_vif_features = []

    for i in range(0, len(features), batch_size):
        batch_features = features[i:i+batch_size]

        # Ensure no missing values untuk VIF calculation
        batch_df = df[batch_features].fillna(df[batch_features].median())

        # Remove constant columns
        variable_cols = []
        for col in batch_features:
            if batch_df[col].nunique() > 1:
                variable_cols.append(col)

        if len(variable_cols) < 2:
            continue

        try:
            # Calculate VIF
            vif_data = pd.DataFrame()
            vif_data["Feature"] = variable_cols
            vif_data["VIF"] = [variance_inflation_factor(batch_df[variable_cols].values, j)
                              for j in range(len(variable_cols))]

            # Flag high VIF features
            high_vif_batch = vif_data[vif_data['VIF'] > 10]['Feature'].tolist()
            high_vif_features.extend(high_vif_batch)

            print(f"Batch {i//batch_size + 1}: Found {len(high_vif_batch)} high VIF features")

        except Exception as e:
            print(f"Error in batch {i//batch_size + 1}: {e}")
            continue

    return list(set(high_vif_features))

# Calculate VIF untuk key feature groups
key_features = [col for col in numerical_cols if any(keyword in col.upper() for keyword in
               ['AMT_', 'DAYS_', 'EXT_SOURCE', 'BUREAU_', 'PREV_', 'PAYMENT_', 'RATIO'])]

print(f"Analyzing VIF for {len(key_features)} key features...")
high_vif_features = calculate_vif_batch(app_train_fe, key_features[:100])  # Limit untuk performance

print(f"Found {len(high_vif_features)} features with VIF > 10")
if high_vif_features:
    print("High VIF features to consider removing:")
    for feature in high_vif_features[:10]:  # Show first 10
        print(f"  - {feature}")

# ==========================================
# STEP 3: CORRELATION-BASED FEATURE REMOVAL
# ==========================================
print("\n📈 STEP 3: Correlation-based Feature Selection...")

# Calculate correlation matrix untuk numerical features
correlation_matrix = app_train_fe[numerical_cols[:100]].corr()  # Limit untuk performance

# Find highly correlated pairs
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = abs(correlation_matrix.iloc[i, j])
        if corr_value > 0.95:  # Very high correlation threshold
            high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], corr_value))

print(f"Found {len(high_corr_pairs)} highly correlated pairs (>0.95)")

# Recommend features to drop (keep the one with better business meaning)
features_to_drop = []
for pair in high_corr_pairs:
    feature1, feature2, corr = pair

    # Business logic untuk memilih feature mana yang di-drop
    if 'GOODS_PRICE' in feature1 and 'CREDIT' in feature2:
        features_to_drop.append(feature1)  # Drop GOODS_PRICE, keep CREDIT
    elif 'GOODS_PRICE' in feature2 and 'CREDIT' in feature1:
        features_to_drop.append(feature2)
    elif 'sum' in feature1 and 'mean' in feature2:
        features_to_drop.append(feature1)  # Keep mean over sum untuk interpretability
    elif 'sum' in feature2 and 'mean' in feature1:
        features_to_drop.append(feature2)
    else:
        features_to_drop.append(pair[1])  # Default: drop second feature

features_to_drop = list(set(features_to_drop))
print(f"Recommending to drop {len(features_to_drop)} highly correlated features")

# ==========================================
# STEP 4: FINAL FEATURE SELECTION
# ==========================================
print("\n🎯 STEP 4: Final Feature Selection...")

# Combine all features to drop
all_features_to_drop = list(set(high_vif_features + features_to_drop))

# Keep essential features regardless of VIF/correlation
essential_features = [
    'TARGET', 'SK_ID_CURR',
    'PAYMENT_TO_INCOME_RATIO', 'LOAN_TO_VALUE_RATIO', 'INCOME_ADEQUACY_RATIO',
    'EXT_SOURCE_WEIGHTED', 'EXT_SOURCE_MEAN',
    'COMPOSITE_RISK_SCORE', 'YEARS_BIRTH', 'YEARS_EMPLOYED',
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
    'CODE_GENDER', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY'
]

# Remove essential features dari drop list
final_features_to_drop = [f for f in all_features_to_drop if f not in essential_features]

print(f"Final recommendation: Drop {len(final_features_to_drop)} features")
print(f"This will reduce dataset from {app_train_fe.shape[1]} to {app_train_fe.shape[1] - len(final_features_to_drop)} columns")

# Apply feature selection
app_train_optimized = app_train_fe.drop(columns=final_features_to_drop, errors='ignore')

print(f"✅ Optimized dataset shape: {app_train_optimized.shape}")

# ==========================================
# STEP 5: CATEGORICAL ENCODING PREPARATION
# ==========================================
print("\n🏷️ STEP 5: Categorical Encoding Preparation...")

# Identify categorical columns
categorical_cols = app_train_optimized.select_dtypes(include=['object']).columns.tolist()
print(f"Found {len(categorical_cols)} categorical columns")

# Separate by cardinality
high_cardinality_cats = []  # For target encoding
low_cardinality_cats = []   # For one-hot encoding

for col in categorical_cols:
    unique_count = app_train_optimized[col].nunique()
    if unique_count > 10:
        high_cardinality_cats.append(col)
    else:
        low_cardinality_cats.append(col)

print(f"High cardinality (>10 unique): {len(high_cardinality_cats)} columns")
print(f"Low cardinality (≤10 unique): {len(low_cardinality_cats)} columns")

# Apply One-Hot Encoding untuk low cardinality
if low_cardinality_cats:
    app_train_encoded = pd.get_dummies(app_train_optimized,
                                      columns=low_cardinality_cats,
                                      prefix=low_cardinality_cats,
                                      drop_first=True)
else:
    app_train_encoded = app_train_optimized.copy()

# Untuk high cardinality, gunakan Label Encoding (Target encoding bisa dilakukan nanti)
label_encoders = {}
for col in high_cardinality_cats:
    le = LabelEncoder()
    app_train_encoded[col + '_ENCODED'] = le.fit_transform(app_train_encoded[col].astype(str))
    label_encoders[col] = le
    # Keep original column untuk reference

print(f"After encoding: {app_train_encoded.shape}")

# ==========================================
# STEP 6: FINAL VALIDATION & SAVE
# ==========================================
print("\n✅ STEP 6: Final Validation...")

# Final checks
print(f"Final dataset shape: {app_train_encoded.shape}")
print(f"Missing values: {app_train_encoded.isnull().sum().sum()}")
print(f"Duplicates: {app_train_encoded.duplicated().sum()}")

# Target distribution
if 'TARGET' in app_train_encoded.columns:
    print(f"Target distribution:\n{app_train_encoded['TARGET'].value_counts(normalize=True)}")

# Feature categories summary
feature_summary = {
    'Total Features': app_train_encoded.shape[1],
    'Numerical Features': len(app_train_encoded.select_dtypes(include=[np.number]).columns),
    'Categorical Features': len(app_train_encoded.select_dtypes(include=['object']).columns),
    'New Advanced Features': len([col for col in app_train_encoded.columns if
                                 any(keyword in col for keyword in ['EXT_SOURCE_', 'RISK_SCORE', 'INTERACTION', 'AGE_GROUP'])]),
    'Features Dropped': len(final_features_to_drop)
}

print("\n📊 Feature Engineering Summary:")
for key, value in feature_summary.items():
    print(f"  {key}: {value}")

# Save optimized dataset
optimized_path = "app_train_optimized_final.csv"
app_train_encoded.to_csv(optimized_path, index=False)
print(f"\n💾 Optimized dataset saved as: {optimized_path}")

print("\n🎉 Feature Engineering Optimization Complete!")
print("Dataset is now ready for modeling with:")
print("  ✅ Advanced engineered features")
print("  ✅ Reduced multicollinearity")
print("  ✅ Proper categorical encoding")
print("  ✅ Optimized feature count")

🔧 Starting Feature Engineering Optimization...
Original dataset shape: (1449, 248)

📊 STEP 1: Creating Advanced Features...
Creating External Source features...
✅ External Source features created
Creating age-based features...
✅ Age-based features created
Creating income-based features...
✅ Income-based features created
Creating interaction features...
✅ Interaction features created
Creating composite risk scores...
✅ Risk scores created
After advanced feature engineering: (1449, 267)

🔍 STEP 2: Multicollinearity Analysis...
Analyzing 244 numerical features for multicollinearity...
Analyzing VIF for 157 key features...
Batch 1: Found 44 high VIF features
Batch 2: Found 47 high VIF features
Found 91 features with VIF > 10
High VIF features to consider removing:
  - BUREAU_CNT_CREDIT_PROLONG_count
  - BUREAU_DAYS_CREDIT_ENDDATE_sum
  - BUREAU_CREDIT_DAY_OVERDUE_min
  - BUREAU_AMT_CREDIT_SUM_max
  - BUREAU_AMT_CREDIT_MAX_OVERDUE_min
  - BUREAU_AMT_CREDIT_MAX_OVERDUE_mean_x
  - PREV_AMT_AP